In [18]:
from openai import OpenAI
import openai, inspect
print("OpenAI class:", OpenAI)
print("openai module path:", openai.__file__)
import re

from dotenv import load_dotenv
_ = load_dotenv()
client = OpenAI()

OpenAI class: <class 'openai.OpenAI'>
openai module path: /Users/mohammadmeftauddin/PycharmProjects/Feb16_BuildingAIAgents/.venv/lib/python3.9/site-packages/openai/__init__.py


In [19]:
class Agent:
    """
    A class representing an AI agent that can engage in conversations using OpenAI's API.
    """

    def __init__(self, system=""):
        """
        Initialize the agent with an optional system message.

        Args:
            system (str): The system message to set the context for the agent.
        """
        self.system = system
        self.messages = []
        if self.system:
            # If a system message is provided, add it to the message history
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        """
        Allow the agent to be called directly with a message.

        Args:
            message (str): The user's input message.

        Returns:
            str: The agent's response to the input message.
        """
        # Add the user's message to the conversation history
        self.messages.append({"role": "user", "content": message})
        # Execute the conversation and get the result
        result = self.execute()
        # Add the assistant's response to the conversation history
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        """
        Execute the conversation by sending the entire conversation history to the OpenAI API.

        Returns:
            str: The content of the model's response.
        """
        # Send the entire conversation history to the OpenAI API
        completion = client.chat.completions.create(
                        model="gpt-4o-mini",
                        temperature=0,
                        messages=self.messages)
        # Return the content of the model's response
        return completion.choices[0].message.content

In [14]:
import sys, pkgutil
print(sys.executable)
print("openai installed?", pkgutil.find_loader("openai") is not None)


/Users/mohammadmeftauddin/PycharmProjects/Feb16_BuildingAIAgents/.venv/bin/python
openai installed? True


In [15]:
from openai import OpenAI
import openai, inspect
print("OpenAI class:", OpenAI)
print("openai module path:", openai.__file__)


OpenAI class: <class 'openai.OpenAI'>
openai module path: /Users/mohammadmeftauddin/PycharmProjects/Feb16_BuildingAIAgents/.venv/lib/python3.9/site-packages/openai/__init__.py


Defining the Agent's Prompt and Behavior
We define a prompt that sets the context and behavior for the agent. The agent operates in a loop of Thought, Action, PAUSE, and Observation, and finally outputs an Answer.

In [20]:
# Define a prompt for the agent
prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate_total_price:
e.g. calculate_total_price: apple: 2, banana: 3
Runs a calculation for the total price based on the quantity and prices of the fruits.

get_fruit_price:
e.g. get_fruit_price: apple
returns the price of the fruit when given its name.

Example session:

Question: What is the total price for 2 apples and 3 bananas?
Thought: I should calculate the total price by getting the price of each fruit and summing them up.
Action: get_fruit_price: apple
PAUSE

Observation: The price of an apple is $1.5.

Action: get_fruit_price: banana
PAUSE

Observation: The price of a banana is $1.2.

Action: calculate_total_price: apple: 2, banana: 3
PAUSE

You then output:

Answer: The total price for 2 apples and 3 bananas is $6.6.
""".strip()

Defining Available Actions
We define functions that the agent can use to perform actions:

In [21]:
# Price lookup for fruits
fruit_prices = {
    "apple": 1.5,
    "banana": 1.2,
    "orange": 1.3,
    "grapes": 2.0
}

# Function to calculate the price of a specific fruit
def get_fruit_price(fruit):
    if fruit in fruit_prices:
        return f"The price of a {fruit} is ${fruit_prices[fruit]}"
    else:
        return f"Sorry, I don't know the price of {fruit}."

# Function to calculate total price based on quantities
def calculate_total_price(fruits):
    total = 0.0
    fruit_list = fruits.split(", ")
    for item in fruit_list:
        fruit, quantity = item.split(": ")
        quantity = int(quantity)
        if fruit in fruit_prices:
            total += fruit_prices[fruit] * quantity
        else:
            return f"Sorry, I don't have the price of {fruit}."
    return f"The total price is ${total:.2f}"

# Mapping actions to functions
known_actions = {
    "get_fruit_price": get_fruit_price,
    "calculate_total_price": calculate_total_price
}

Creating the Agent Instance
We create an instance of the Agent class with the defined prompt.

In [23]:
react_agent = Agent(system=prompt)

In [24]:
# Run a query
action_re = re.compile(r'^Action: (\w+): (.*)$')   # python regular expression to select action

def query(question):
    bot = Agent(prompt)
    result = bot(question)
    print(result)
    actions = [
        action_re.match(a)
        for a in result.split('\n')
        if action_re.match(a)
    ]
    if actions:
        action, action_input = actions[0].groups()
        if action not in known_actions:
            raise Exception(f"Unknown action: {action}: {action_input}")
        print(f" -- running {action} {action_input}")
        observation = known_actions[action](action_input)
        print("Observation:", observation)
    else:
        return

In [25]:
query("What is the price of a banana?")

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [26]:
from openai import OpenAI

client = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

resp = client.chat.completions.create(
    model="gpt-oss:120b-cloud",   # cloud-backed via Ollama
    messages=[{"role": "user", "content": "Explain JWT audience validation in Spring Security."}],
)

print(resp.choices[0].message.content)

## JWT Audience (`aud`) Validation in Spring Security  
Spring Security (the part that implements an **OAuth 2.0 Resource Server**) can decode, verify and turn a JSON Web Token (JWT) into an `Authentication` object.  
One of the checks you normally want to run on a JWT is **audience validation** – i.e. “Is this token intended for *my* API?”  

Below you will find:

| Section | What it covers |
|---------|-----------------|
| 1️⃣ What the `aud` claim is and why you care | Conceptual overview |
| 2️⃣ How Spring Security does validation by default | Built‑in validators |
| 3️⃣ Configuring audience validation with **Boot properties** | “Zero‑code” approach |
| 4️⃣ Fine‑grained custom audience validation (Java config) | Full‑control, multiple audiences, etc. |
| 5️⃣ Putting it together with `HttpSecurity` | Resource‑server DSL |
| 6️⃣ Common pitfalls & troubleshooting | “Gotchas” you’ll run into |
| 7️⃣ Testing the audience validator | `MockMvc` + `@JwtTest` convenience |
| 8️⃣ Full example

In [4]:
import os
from langchain_openai import ChatOpenAI

# Fake key because the OpenAI client requires it (Ollama ignores it completely)
os.environ["OPENAI_API_KEY"] = "ollama"

# Point the OpenAI-compatible client to Ollama instead of api.openai.com
os.environ["OPENAI_BASE_URL"] = "http://localhost:11434/v1"

# Create the LLM using Ollama's model name
llm = ChatOpenAI(
    model="gpt-oss:120b-cloud",    # using the 8B model
    temperature=0.2,
)

# Quick test call (optional)
resp = llm.invoke("Say 'hello from PyCharm + Ollama + LangGraph setup!'")
print(resp.content)

hello from PyCharm + Ollama + LangGraph setup!


In [5]:
import ssl
print(ssl.OPENSSL_VERSION)


LibreSSL 2.8.3
